# 04 - Transformers (Teoría)

Transformers nació con el paper Attention is All You Need. El grupo de NLP de Harvard creó una guía anotando el paper con la implementación en PyTorch. Vamos a intentar simplificar un poco las cosas e introducir los conceptos uno por uno para que sea más fácil de entender.

### Una mirada de alto nivel

Empecemos por ver el modelo como una caja negra. En una aplicación de traducción automática, tomaría una frase en un idioma y la traduciría a otro.

![transformers_exp_1](../images/transformers_exp_1.png)

En las tripas de Optimus Prime, hay un componente de codificación (encoder), un componente de descodificación (decoder) y conexiones entre ellos.

![transformers_exp_2](../images/transformers_exp_2.png)

El encoder es una pila de codificadores (en el paper se apilan seis de ellos uno encima de otro; no hay nada mágico en el número seis, se puede experimentar con otras disposiciones). El decoder es una pila de descodificadores del mismo número.

![transformers_exp_3](../images/transformers_exp_3.png)

Todos los encoders tienen la misma estructura, aunque no comparten sus pesos internos. Cada uno se divide en dos subcapas:

![transformers_exp_4](../images/transformers_exp_4.png)

Las entradas del encoder pasan primero por una capa de "auto-atención" (self-attention), que ayuda al encoder a mirar otras palabras de la frase de entrada mientras codifica una palabra concreta. Más adelante veremos con más detalle la auto-atención.

Las salidas de la capa de auto-atención se envían a una red neuronal feed-forward. Exactamente la misma red feed-forward se aplica independientemente a cada posición.

El decoder tiene estas dos capas, pero entre ellas hay una capa de atención que ayuda al decoder a centrarse en las partes relevantes de la frase de entrada, algo similar a lo que hace la atención en los modelos seq2seq.

![transformers_exp_5](../images/transformers_exp_5.png)

### Introduciendo los tensores

Ahora que hemos visto los principales bloques del modelo, vamos a empezar a ver los distintos vectores/tensores y cómo fluyen entre estos componentes para convertir la entrada de un modelo entrenado en una salida.

Como ocurre en las aplicaciones NLP en general, empezamos convirtiendo cada palabra de entrada en un vector mediante un algoritmo de incrustado (word embeddings).

![transformers_exp_6](../images/transformers_exp_6.png)

Cada palabra está incrustada en un vector de tamaño 512. Representaremos esos vectores con estos simples recuadros.


El embebido sólo se produce en el primer encoder, el situado más abajo. La abstracción que es común a todos los codificadores es que reciben una lista de vectores cada uno del tamaño 512. En el encoder de la siguiente imagen eso serían las palabras embebidas, pero en otros codificadores, sería la salida del codificador que está directamente debajo. El tamaño de esta lista es un hiperparámetro que podemos establecer, sería básicamente la longitud de la frase más larga de nuestro conjunto de datos de entrenamiento.


Después de embeber las palabras en nuestra secuencia de entrada, cada una de ellas fluye a través de cada una de las dos capas del codificador.

![transformers_exp_7](../images/transformers_exp_7.png)

Aquí empezamos a ver una propiedad clave del Transformer, que es que la palabra en cada posición fluye a través de su propio camino en el codificador. Existen dependencias entre estos caminos en la capa de auto-atención. Sin embargo, la capa feed-forward no tiene esas dependencias y, por tanto, los distintos caminos pueden ejecutarse en paralelo mientras fluyen a través de la capa feed-forward.

A continuación, cambiaremos el ejemplo a una frase más corta y veremos lo que ocurre en cada subcapa del encoder.

### Ahora encoder

Como ya hemos dicho, un encoder recibe una lista de vectores como entrada. Procesa esta lista pasando estos vectores a una capa de "auto-atención", luego a una red neuronal feed-forward y, por último, envía la salida hacia arriba al siguiente encoder.

![transformers_exp_8](../images/transformers_exp_8.png)

La palabra en cada posición pasa por un proceso de auto-atención. A continuación, cada una de ellas pasa por una red neuronal feed-forward, exactamente la misma red, pero cada vector fluye por ella por separado

### Autoatención a alto nivel


No nos engañemos con la palabra "auto-atención", como si fuera un concepto con el que todo el mundo debería estar familiarizado. Todo está explicado en el paper "Attention is All You Need". Veamos cómo funciona.


Supongamos que queremos traducir la siguiente frase:

"The animal didn't cross the street because it was too tired".

¿A qué se refiere "it" en esta frase? ¿Se refiere a la calle o al animal? Es una pregunta sencilla para un humano, pero no tanto para un algoritmo.

Cuando el modelo está procesando la palabra "it", la auto-atención le permite asociar "it" con "animal".

A medida que el modelo procesa cada palabra, cada posición en la secuencia de entrada, la auto-atención le permite buscar pistas en otras posiciones de la secuencia de entrada que le ayuden a codificar mejor esta palabra.


![transformers_exp_9](../images/transformers_exp_9.png)

Como estamos codificando la palabra "it" en el encoder nº5, el encoder superior de la pila, parte del mecanismo de atención se estaba centrando en "The animal", e incorporó una parte de su representación a la codificación de "it".

### Auto-atención en detalle


Veamos primero cómo calcular la auto-atención utilizando vectores y, a continuación, cómo se implementa realmente, utilizando matrices.

**El primer paso** para calcular la auto-atención es crear tres vectores a partir de cada uno de los vectores de entrada del encoder, en este caso, el embebido de cada palabra. Así, para cada palabra, creamos un vector Query, un vector Key y un vector Value. Estos vectores se crean multiplicando el embebido por tres matrices que entrenamos durante el proceso de formación.

Estos nuevos vectores tienen una dimensión menor que el vector embebido. Su dimensionalidad es de 64, mientras que los vectores de entrada/salida del embebido y del encoder tienen una dimensionalidad de 512. No TIENEN que ser más pequeños, esta es una elección de arquitectura para hacer el cómputo de la multi-atención (multiheaded attention) en su mayoría constante.

![transformers_exp_10](../images/transformers_exp_10.png)

Al multiplicar $x1$ por la matriz de pesos $WQ$ se obtiene $q1$, el vector de Query asociado a esa palabra. Al final creamos una proyección de Query, una de Key y una de Value de cada palabra de la frase de entrada.

¿Qué son los vectores Query, Key y Value?

Son abstracciones útiles para calcular y pensar sobre la atención. Veamos como se calcula la atención para entenderlo.

**El segundo paso** para calcular la auto-atención es calcular una puntuación. Digamos que estamos calculando la auto-atención para la primera palabra de este ejemplo, "Thinking". Tenemos que puntuar cada palabra de la frase de entrada con respecto a esta palabra. La puntuación determina el grado de atención que debemos prestar a otras partes de la frase de entrada cuando codificamos una palabra en una posición determinada.

La puntuación se calcula tomando el producto escalar del vector Query por el vector Key de la palabra correspondiente. Así, si estamos procesando la auto-atención de la palabra en la posición nº 1, la primera puntuación sería el producto punto de $q1$ y $k1$. La segunda puntuación sería el producto escalar de $q1$ y $k2$.

![transformers_exp_11](../images/transformers_exp_11.png)

**El tercer y cuarto paso** consisten en dividir las puntuaciones por 8, la raíz cuadrada de la dimensión de los vectores Key utilizados en el paper: 64. Esto nos lleva a tener gradientes más estables. Podría haber otros valores posibles aquí, pero este es el predeterminado. Luego pasar el resultado a través de una operación softmax. Softmax normaliza las puntuaciones para que sean todas positivas y sumen 1.

![transformers_exp_12](../images/transformers_exp_12.png)

Esta puntuación softmax determina cuánto se expresará cada palabra en esta posición. Está claro que la palabra en esta posición tendrá la puntuación softmax más alta, pero a veces es útil atender a otra palabra que sea relevante para la palabra actual.

**El quinto paso** consiste en multiplicar cada vector Value por la puntuación softmax, para luego sumarlos. La intuición aquí es mantener intactos los valores de la(s) palabra(s) en las que queremos centrarnos, y ahogar las palabras irrelevantes, multiplicándolas por números minúsculos como 0,001, por ejemplo.


**El sexto paso** consiste en sumar los vectores de valores ponderados. Esto produce la salida de la capa de auto-atención en esta posición, para la primera palabra.

![transformers_exp_13](../images/transformers_exp_13.png)

Con esto concluye el cálculo de la auto-atención. El vector resultante es el que podemos enviar a la red neuronal feed-forward. En la implementación real, sin embargo, este cálculo se realiza en forma de matriz para un procesamiento más rápido. Veámoslo ahora que hemos visto la intuición del cálculo a nivel de palabra.

## Cálculo de la matriz de autoatención


**El primer paso** consiste en calcular las matrices de Query, Key y Value. Para ello, empaquetamos nuestros embebidos en una matriz X y la multiplicamos por las matrices de pesos que hemos entrenado (WQ, WK, WV).

![transformers_exp_14](../images/transformers_exp_14.png)

Cada fila de la matriz X corresponde a una palabra de la frase de entrada. Volvemos a ver la diferencia de tamaño entre el vector embebido, 512, o 4 casillas en la figura, y los vectores $q$/$k$/$v$, 64, o 3 casillas en la figura.


**Por último**, como se trata de matrices, podemos condensar los **pasos dos a seis en una fórmula** para calcular los resultados de la capa de auto-atención.

![transformers_exp_15](../images/transformers_exp_15.png)

## La bestia de muchas cabezas

El paper perfecciona la capa de auto-atención añadiendo un mecanismo denominado multi-atención (multi-headed attention). Esto mejora el rendimiento de la capa de atención de dos maneras:

1. Amplía la capacidad del modelo para centrarse en distintas posiciones. Sí, en el ejemplo anterior, $z1$ contiene un poco de cualquier otra codificación, pero podría estar dominada por la propia palabra. Si estamos traduciendo una frase como "The animal didn’t cross the street because it was too tired", sería útil saber a qué palabra se refiere "it".

2. Esto proporciona a la capa de atención múltiples "subespacios de representación". Como veremos a continuación, con la multi-atención tenemos no sólo uno, sino múltiples conjuntos de matrices de pesos Query/Key/Value, el Transformer utiliza ocho cabezales de atención, por lo que acabamos con ocho conjuntos para cada encoder/decoder. Cada uno de estos conjuntos se inicializa aleatoriamente. Después del entrenamiento, cada conjunto se utiliza para proyectar las incrustaciones de entrada, o vectores de codificadores/decodificadores inferiores, en un subespacio de representación diferente.

![transformers_exp_16](../images/transformers_exp_16.png)

Con multi-atención, mantenemos las matrices de pesos Q/K/V separadas para cada cabezal, lo que da lugar a matrices Q/K/V diferentes. Como hicimos antes, multiplicamos X por las matrices WQ/WK/WV para producir matrices Q/K/V.

Si hacemos el mismo cálculo de auto-atención que esbozamos antes, sólo que ocho veces diferentes con matrices de peso distintas, acabamos con ocho matrices Z diferentes

![transformers_exp_17](../images/transformers_exp_17.png)

Esto nos plantea un pequeño reto. La capa feed-forward no espera ocho matrices, sino una sola, un vector por cada palabra. Así que necesitamos una forma de condensar esas ocho matrices en una sola.

¿Cómo lo hacemos? Concatenamos las matrices y las multiplicamos por una matriz de pesos adicional WO.

![transformers_exp_18](../images/transformers_exp_18.png)

Eso es más o menos todo lo que hay en la auto-atención multicabeza. Es un buen puñado de matrices. Vamos a ponerlos todos en una gráfica para que podamos verlos en un solo lugar.

![transformers_exp_19](../images/transformers_exp_19.png)

Ahora que hemos hablado de las cabezas de atención, volvamos a nuestro ejemplo anterior para ver dónde se centran las diferentes cabezas de atención cuando codificamos la palabra "it" en nuestra frase de ejemplo:

![transformers_exp_20](../images/transformers_exp_20.png)

Cuando codificamos la palabra "it", una cabeza de atención se centra sobre todo en "The animal", mientras que otra se centra en "tired"; en cierto sentido, la representación del modelo de la palabra "it" incorpora parte de la representación tanto de "animal" como de "tired".


Sin embargo, si añadimos todas las cabezas de atención a la imagen, las cosas pueden ser más difíciles de interpretar:

![transformers_exp_21](../images/transformers_exp_21.png)

# Representación del orden de la secuencia mediante codificación posicional

Lo que falta en el modelo tal y como lo hemos descrito hasta ahora es una forma de tener en cuenta el orden de las palabras en la secuencia de entrada.

Para ello, el transformer añade un vector a cada embebido de entrada. Estos vectores siguen un patrón específico que el modelo aprende y que le ayuda a determinar la posición de cada palabra o la distancia entre las distintas palabras de la secuencia. La intuición aquí es que añadir estos valores a los embebidos proporciona distancias significativas entre los vectores una vez que se proyectan en vectores Q/K/V y durante la atención en el producto escalar.

![transformers_exp_22](../images/transformers_exp_22.png)

Para dar al modelo una idea del orden de las palabras, añadimos vectores de codificación posicional, cuyos valores siguen un patrón específico.


Si suponemos que el embebido tiene una dimensionalidad de 4, las codificaciones posicionales reales tendrían este aspecto:

![transformers_exp_23](../images/transformers_exp_23.png)

¿Qué aspecto podría tener este patrón?

En la siguiente figura, cada fila corresponde a una codificación posicional de un vector. Así, la primera fila sería el vector que añadiríamos al embebido de la primera palabra de una secuencia de entrada. Cada fila contiene 512 valores, cada uno con un valor entre 1 y -1. 

![transformers_exp_24](../images/transformers_exp_24.png)

Este es un ejemplo real de codificación posicional para 20 palabras (filas) con un tamaño de embebido de 512 (columnas). Se puede ver que aparece partido por la mitad en el centro. Esto se debe a que los valores de la mitad izquierda son generados por una función (que utiliza el seno), y la mitad derecha es generada por otra función (que utiliza el coseno). Luego se concatenan para formar cada uno de los vectores de codificación posicional.
La fórmula de la codificación posicional se describe en el paper (sección 3.5). 

# Los residuos

Un detalle en la arquitectura del codificador que debemos mencionar antes de seguir adelante, es que cada subcapa (auto-atención, feed-forward) en cada encoder tiene una conexión residual a su alrededor, y es seguida por un paso de normalización de capas.

![transformers_exp_25](../images/transformers_exp_25.png)

Si visualizáramos los vectores y la operación aññadir-normalizar asociada a la auto-atención, se vería así:

![transformers_exp_26](../images/transformers_exp_26.png)

Lo mismo ocurre con las subcapas del decoder. Si pensamos en un Transformer de 2 encoders y decoders apilados, sería algo así:

![transformers_exp_27](../images/transformers_exp_27.png)

# El lado del decoder

Ahora que hemos estudiado la mayoría de los conceptos del lado del encoder, ya sabemos cómo funcionan los componentes de los decoders. Pero veamos cómo funcionan juntos.

El encoder comienza procesando la secuencia de entrada. A continuación, la salida del encoder superior se transforma en un conjunto de vectores de atención K y V. Cada decoder los utiliza en su capa de "atención encoder-decoder", que ayuda al decoder a centrarse en los lugares adecuados de la secuencia de entrada:

![transformers_exp_28](../images/transformers_exp_28.gif)

Una vez finalizada la fase de codificación, comenzamos la fase de descodificación. Cada paso de la fase de descodificación da salida a un elemento de la secuencia de salida, la frase traducida al inglés en este caso.

Los pasos siguientes repiten el proceso hasta que se alcanza un símbolo especial que indica que el decoder del transformer ha completado su salida. La salida de cada paso se envía al decoder inferior en el siguiente paso temporal, y los decoders burbujean sus resultados de descodificación igual que hicieron los encoders. Y al igual que hicimos con las entradas del encoder, embebemos y añadimos codificación posicional a esas entradas del decoder para indicar la posición de cada palabra.

![transformers_exp_29](../images/transformers_exp_29.gif)

Las capas de auto-atención del decoder funcionan de forma ligeramente distinta a las del encoder:

En el decoder, a la capa de auto-atención sólo se le permite atender a posiciones anteriores en la secuencia de salida. Esto se hace enmascarando las posiciones futuras, poniéndolas a -inf, antes del paso softmax en el cálculo de auto-atención.

La capa "Atención Encoder-Decoder" funciona igual que la multi-atención, excepto que crea su matriz Query a partir de la capa inferior, y toma la matriz Key y Value de la salida de la pila del encoder.

# Capa final lineal y Softmax

La pila del decoder produce un vector de floats. ¿Cómo lo convertimos en una palabra? De eso se encarga la última capa lineal, a la que sigue una capa Softmax.

La capa lineal es una sencilla red neuronal totalmente conectada que proyecta el vector producido por la pila de decoders en un vector mucho mayor, llamado vector logit.

Supongamos que nuestro modelo conoce 10.000 palabras únicas en inglés, el "vocabulario de salida" de nuestro modelo, que ha aprendido de su conjunto de datos de entrenamiento. Esto haría que el vector logit tuviera 10.000 celdas de ancho, cada una de las cuales correspondería a la puntuación de una palabra única. Así es como interpretamos la salida del modelo seguido por la capa Lineal.

A continuación, la capa softmax convierte esas puntuaciones en probabilidades, todas positivas, todas suman 1. Se elige la celda con la probabilidad más alta, y la palabra asociada a ella se produce como la salida para este paso de tiempo.

![transformers_exp_30](../images/transformers_exp_30.png)

<br>
<br>
<br>
<br>

# Entrenamiento

Ahora que hemos cubierto todo el proceso de paso adelante a través de un Transformer entrenado, sería útil echar un vistazo al propio entrenamiento del modelo.

Durante el entrenamiento, un modelo no entrenado pasaría exactamente por el mismo proceso. Pero como lo estamos entrenando en un conjunto de datos de entrenamiento etiquetados, podemos comparar su resultado con el resultado correcto real.

Para visualizarlo, supongamos que nuestro vocabulario de salida sólo contiene seis palabras ("a", "am", "i", "thanks", "student" y "<eos>" (abreviatura de "end of sentence")).

![transformers_exp_31](../images/transformers_exp_31.png)

El vocabulario de salida de nuestro modelo se crea en la fase de preprocesamiento, antes incluso de empezar el entrenamiento.

Una vez definido nuestro vocabulario de salida, podemos utilizar un vector de la misma anchura para indicar cada palabra de nuestro vocabulario. Esto también se conoce como codificación one-hot. Así, por ejemplo, podemos indicar la palabra "am" utilizando el siguiente vector:

![transformers_exp_32](../images/transformers_exp_32.png)

Tras este resumen, hablemos de la función de pérdida del modelo, la métrica que optimizamos durante la fase de entrenamiento para llegar a un modelo entrenado y, con suerte, asombrosamente preciso.

# La función de pérdida

Supongamos que estamos entrenando nuestro modelo. Digamos que es nuestro primer paso en la fase de entrenamiento y que lo estamos entrenando con un ejemplo sencillo: traducir "merci" por "thanks".

Esto significa que queremos que la salida sea una distribución de probabilidad que indique la palabra "thanks". Pero como este modelo aún no está entrenado, es poco probable que eso ocurra todavía.

![transformers_exp_33](../images/transformers_exp_33.png)

Dado que los parámetros del modelo, pesos, se inicializan aleatoriamente, el modelo no entrenado produce una distribución de probabilidad con valores arbitrarios para cada celda/palabra. Podemos compararla con la salida real y, a continuación, ajustar todos los pesos del modelo mediante retropropagación para que la salida se acerque más a la salida deseada.


¿Cómo se comparan dos distribuciones de probabilidad? Simplemente restamos una de la otra. Para más detalles, consulte la entropía cruzada y la divergencia de Kullback-Leibler.

Pero tenga en cuenta que se trata de un ejemplo demasiado simplificado. Para ser más realistas, utilizaremos una frase de más de una palabra. Por ejemplo: "je suis étudiant" y salida esperada: "i am a student". Lo que esto significa realmente es que queremos que nuestro modelo produzca sucesivamente distribuciones de probabilidad donde:

+ Cada distribución de probabilidad está representada por un vector de anchura vocab_size, 6 en nuestro ejemplo de juguete, pero más realista un número como 30.000 o 50.000.
+ La primera distribución de probabilidad tiene la probabilidad más alta en la celda asociada a la palabra "i".
+ La segunda distribución de probabilidad tiene la mayor probabilidad en la celda asociada a la palabra "am".
+ Y así sucesivamente, hasta que la quinta distribución de salida indica el símbolo "<eos>", que también tiene asociada una celda del vocabulario de 10.000 elementos.

![transformers_exp_34](../images/transformers_exp_34.png)

Las distribuciones de probabilidad objetivo con las que entrenaremos nuestro modelo en el ejemplo de entrenamiento para una frase de muestra.


Después de entrenar el modelo durante el tiempo suficiente en un conjunto de datos suficientemente grande, esperamos que las distribuciones de probabilidad producidas tengan este aspecto:

![transformers_exp_35](../images/transformers_exp_35.png)

Es de esperar que, tras el entrenamiento, el modelo produzca la traducción correcta que esperamos. Por supuesto, no hay ninguna indicación real de si esta frase formaba parte del conjunto de datos de entrenamiento, véase validación cruzada. Cada posición recibe un poco de probabilidad, incluso si es poco probable que sea la salida de ese paso de tiempo, que es una propiedad muy útil de softmax que ayuda al proceso de formación.

Ahora, como el modelo produce las salidas de una en una, podemos suponer que el modelo está seleccionando la palabra con la probabilidad más alta de esa distribución de probabilidad y desechando el resto. Esa es una forma de hacerlo, llamada decodificación codiciosa (greedy decoding). Otra forma de hacerlo sería quedarnos, por ejemplo, con las dos primeras palabras, por ejemplo, "I" y "a", y, en el siguiente paso, ejecutar el modelo dos veces: una vez suponiendo que la primera posición de salida es la palabra "I" y otra vez suponiendo que la primera posición de salida es la palabra "a". Repetimos esto para las posiciones 2, 3...etc. Este método se llama "búsqueda de haces" (beam search), donde en nuestro ejemplo, beam_size era dos, lo que significa que en todo momento se guardan en memoria dos hipótesis parciales, traducciones inacabadas, y top_beams es también dos, lo que significa que devolveremos dos traducciones. Ambos son hiperparámetros con los que se puede experimentar.